In [12]:
#for now very loose on the requirments later we need a requirments.txt
import torch
import intel_extension_for_pytorch as ipex

In [13]:
from transformers import AutoProcessor, AutoModel
import scipy

processor = AutoProcessor.from_pretrained("suno/bark")
model = AutoModel.from_pretrained("suno/bark").to('xpu')

C:\Users\intel\miniforge3\envs\openvino\Lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
C:\Users\intel\miniforge3\envs\openvino\Lib\site-packages\transformers\models\encodec\modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [14]:
inputs = processor(
    text=["Dramatic piano music"],
    return_tensors="pt",
)
inputs = {k:v.to('xpu') for k,v in inputs.items()}

In [ ]:
speech_values = model.generate(**inputs, do_sample=True).cpu()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [ ]:
from IPython.display import Audio

sampling_rate = model.generation_config.sample_rate
Audio(speech_values.cpu().numpy().squeeze(), rate=sampling_rate)


In [8]:
import scipy

sampling_rate = model.generation_config.sample_rate
scipy.io.wavfile.write("bark_out.wav", rate=sampling_rate, data=speech_values.numpy().squeeze())